In [15]:
import pandas as pd

In [21]:
#df = pd.read_csv("processed_data.csv")
df = pd.read_csv('processed_data_test.csv')
df_dropped = df.drop(['Timestamp_x', 'Timestamp_y', 'Timestamp_dt', 'OpenDate', 'Device_List', 'CustomerID_y', 'Timestamp_y', 'Actions'], axis=1)

In [17]:
cols = ["Channel", "MCC_Group", "Type", "Day_of_Week", "Type_dst"]

for i in cols:
    df_dropped[i] = df_dropped[i].astype("category")
for i in cols:
    df_dropped[i] = df_dropped[i].cat.codes

In [22]:
df_dropped.dtypes

Unnamed: 0.1                   int64
Unnamed: 0                     int64
TxnID                         object
Online                         int64
CustomerID                    object
SrcAccount                    object
DstAccount                    object
Channel                       object
MCC_Group                     object
Amount                       float64
CustomerID_src                object
AccountID                     object
Type                          object
Balance                      float64
Limit                        float64
CustomerID_dst                object
AccountID_dst                 object
Type_dst                      object
Balance_dst                  float64
Limit_dst                    float64
OpenDate_dst                  object
Age                          float64
Tenure                       float64
CreditScore                  float64
HomeCity                      object
AnnualSalary                 float64
Device_Count                   int64
C

In [23]:

df_dropped ['Timestamp'] = df_dropped['Timestamp'].apply(lambda x: int(x[0:2]) + int(float(x[3:5])/60))
df_dropped['Timestamp']

0          9
1         16
2         17
3         18
4         22
          ..
151582    17
151583    22
151584    22
151585    12
151586    10
Name: Timestamp, Length: 151587, dtype: int64

In [24]:
# Selected features for our model
labels = ['TxnID', 'Timestamp_dt', 'CustomerID', 'Online', 'Channel', 'MCC_Group', 'Amount',
              'Type', 'Balance', 'Limit', 'Type_dst', 'Balance_dst',
              'Limit_dst', 'Age', 'Tenure', 'CreditScore', 'AnnualSalary',
              'Device_Count', 'Action_Count', 'Login_Count', 'Logout_Count',
              'Account_View_Count', 'Payment_Count', 'Transfer_Count',
              'Day_of_Week', 'Timestamp', 'in_home_city', 'prev_trans_in_city',
              'times_device_used', 'unique_devices_past', 'days_since_open',
              'days_since_open_dst', 'time_since_last_txn_hours',
              'xacts_earlier_today', 'avg_amount_past', 'amount_ratio',
              'amount_dev', 'mcc_group_prev_count']

# Get lagged variables
my_df = df[labels]
my_df = my_df.sort_values(by=['CustomerID', 'Timestamp_dt'])
my_df = my_df.set_index('TxnID')
shifted = my_df.groupby('CustomerID').shift(periods=1)
my_df = my_df.join(shifted['Amount'].rename('LaggedAmt'))
my_df = my_df.join(shifted['Action_Count'].rename('LaggedActCt'))
my_df = my_df.join(shifted['in_home_city'].rename('LaggedHomeCity'))
my_df = my_df.join(shifted['Payment_Count'].rename('LaggedPayCt'))
my_df = my_df.join(shifted['Transfer_Count'].rename('LaggedTransCt'))
my_df['Lagged50Amt'] = my_df.groupby('CustomerID')['Amount'].rolling(window=50).mean().reset_index(level=0, drop=True)
my_df['Lagged50ActCt'] = my_df.groupby('CustomerID')['Action_Count'].rolling(window=50).mean().reset_index(level=0, drop=True)
my_df['Lagged50HomeCity'] = my_df.groupby('CustomerID')['in_home_city'].rolling(window=50).mean().reset_index(level=0, drop=True)
my_df['Lagged50PayCt'] = my_df.groupby('CustomerID')['Payment_Count'].rolling(window=50).mean().reset_index(level=0, drop=True)
my_df['Lagged50TransCt'] = my_df.groupby('CustomerID')['Transfer_Count'].rolling(window=50).mean().reset_index(level=0, drop=True)
my_df['Lagged25Amt'] = my_df.groupby('CustomerID')['Amount'].rolling(window=25).mean().reset_index(level=0, drop=True)
my_df['Lagged25ActCt'] = my_df.groupby('CustomerID')['Action_Count'].rolling(window=25).mean().reset_index(level=0, drop=True)
my_df['Lagged25HomeCity'] = my_df.groupby('CustomerID')['in_home_city'].rolling(window=25).mean().reset_index(level=0, drop=True)
my_df['Lagged25PayCt'] = my_df.groupby('CustomerID')['Payment_Count'].rolling(window=25).mean().reset_index(level=0, drop=True)
my_df['Lagged25TransCt'] = my_df.groupby('CustomerID')['Transfer_Count'].rolling(window=25).mean().reset_index(level=0, drop=True)
my_df['Lagged100Amt'] = my_df.groupby('CustomerID')['Amount'].rolling(window=100).mean().reset_index(level=0, drop=True)
my_df['Lagged100ActCt'] = my_df.groupby('CustomerID')['Action_Count'].rolling(window=100).mean().reset_index(level=0, drop=True)
my_df['Lagged100HomeCity'] = my_df.groupby('CustomerID')['in_home_city'].rolling(window=100).mean().reset_index(level=0, drop=True)
my_df['Lagged100PayCt'] = my_df.groupby('CustomerID')['Payment_Count'].rolling(window=100).mean().reset_index(level=0, drop=True)
my_df['Lagged100TransCt'] = my_df.groupby('CustomerID')['Transfer_Count'].rolling(window=100).mean().reset_index(level=0, drop=True)
my_df['Lagged50AmtDiff'] = my_df['Amount'] - my_df['Lagged50Amt']
my_df['Lagged50TransDiff'] = my_df['Transfer_Count'] - my_df['Lagged50TransCt']
my_df['Lagged50PaymentDiff'] = my_df['Payment_Count'] - my_df['Lagged50PayCt']

cols_to_keep = ['LaggedAmt', 'LaggedActCt', 'LaggedHomeCity', 'LaggedPayCt', 'LaggedTransCt',
                'Lagged50Amt', 'Lagged50ActCt', 'Lagged50HomeCity', 'Lagged50PayCt', 'Lagged50TransCt',
                'Lagged25Amt', 'Lagged25ActCt', 'Lagged25HomeCity', 'Lagged25PayCt', 'Lagged25TransCt',
                'Lagged100Amt', 'Lagged100ActCt', 'Lagged100HomeCity', 'Lagged100PayCt', 'Lagged100TransCt',
                'Lagged50AmtDiff', 'Lagged50TransDiff', 'Lagged50PaymentDiff']
my_df = my_df[cols_to_keep]
my_df.to_csv('LaggedTest.csv')
my_df
#>>> shifted = df.groupby(level="grp").shift(-1)
#>>> df.join(shifted.rename(columns=lambda x: x+"_lag"))

,LaggedAmt,LaggedActCt,LaggedHomeCity,LaggedPayCt,LaggedTransCt,Lagged50Amt,Lagged50ActCt,Lagged50HomeCity,Lagged50PayCt,Lagged50TransCt,...,Lagged25PayCt,Lagged25TransCt,Lagged100Amt,Lagged100ActCt,Lagged100HomeCity,Lagged100PayCt,Lagged100TransCt,Lagged50AmtDiff,Lagged50TransDiff,Lagged50PaymentDiff
TxnID,,,,,,,,,,,,,,,,,,,,,
c77cfd37-2a4f-41a2-ae45-696aaa2ec4e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1e3da79d-891c-4769-af85-4b9bfc2aab7c,26.436533,0.0,False,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b1e5643f-3fee-41e6-b003-a360810dfa3d,39.520422,0.0,False,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540ab70c-c05e-4a68-bddb-fb176e08bf62,82.786723,0.0,False,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2e1ccb91-e11f-46a5-886f-6f9a23550d0a,12.247675,0.0,False,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743db2e8-34dd-41ce-8856-fa47df3285d7,29.133643,0.0,False,0.0,0.0,52.035490,0.24,0.06,0.04,0.02,...,0.08,0.04,52.383930,0.15,0.04,0.03,0.01,38.573816,-0.02,-0.04
0bca8ddb-3a15-40b2-884f-a1aa22dc4645,90.609306,0.0,False,0.0,0.0,50.112673,0.34,0.08,0.06,0.04,...,0.12,0.08,51.065912,0.20,0.05,0.04,0.02,-93.077076,0.96,0.94
03abc94f-9c96-4d57-beb8-f4cd58f31478,-42.964403,5.0,True,1.0,1.0,45.690015,0.44,0.10,0.08,0.06,...,0.16,0.12,50.219620,0.25,0.06,0.05,0.03,-65.142309,0.94,0.92


In [ ]:
#df_dropped.to_csv("final_train.csv", index=False)
#df_dropped.to_csv('final_test.csv', index=False)